# Getting basins for all NVE discharge-measuring stations
This notebook assumes that NVE info has been downloaded using the get_nve_data.ipynb notebook in this folder

## Loading required packages and getting credentials to connect to the database

In [1]:
import requests
import json
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from datetime import datetime
from datetime import timedelta
from fabric import Connection
import psycopg2
import gmaps
import pandas.io.sql as sqlio
import getpass
from io import StringIO
import paramiko
import sys
from time import sleep
import gdal
import numpy as np

sys.path.insert(0, "/home/jovyan/watexr/PROGNOS/")
from prognos_tools.gce_light import gce_api as gce

key = getpass.getpass('mobiserver password: ')
cloudKey = getpass.getpass('vault password: ')
#Querying necessary tokens
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
        with db.cursor() as cursor :
            cursor.execute(query)
            if fetch:
                result = sqlio.read_sql_query(query, db)
                return result
            
gmapsKey = query('''select niva.getToken('gmaps','{}');'''.format(cloudKey)).iloc[0,0]
sshKey = query('''select niva.getToken('geonorway_ssh_key','{}');'''.format(cloudKey)).iloc[0,0]
cloudKey = json.loads(query('''select niva.getToken('gce_access','{}');'''.format(cloudKey)).iloc[0,0])
gmaps.configure(gmapsKey)
not_really_a_file = StringIO(sshKey)
private_key = paramiko.RSAKey.from_private_key(not_really_a_file)
del key,sshKey

mobiserver password:  ·······
vault password:  ···············


## Starting the instance containing the database using the google compute engine api

In [2]:
#Check status of instance
properties = {'project'      : 'nivacatchment',
             'zone'         : 'europe-north1-a',
             'instanceType' : "n1-standard-4",
             'instanceName' : "dtm10",
             'username'     : "jose-luis",
             }

cloud = gce(properties, cloudKey)
del cloudKey

#Getting instance info
cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
info = cloud.get('custom')
display(info['status'])
#If instance is stopped, start it
if info['status'] != 'RUNNING':
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}/start'''
    info = cloud.post('custom')
    display(info['status'])
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
    info = cloud.get('custom')
    while info['status'] != 'RUNNING':
           sleep(2)
           info = cloud.get('custom')
          
geonorway = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']    
    
display("IP of the instance containing the database {}:".format(geonorway))

#The config below can be used to connect to the instance using fabric's Connection
config =  {'host' : geonorway, 'user': 'jose-luis', 'connect_kwargs': {'pkey': private_key } }

'RUNNING'

'IP of the instance containing the database 35.228.213.48:'

## Getting the coordinates of NVE's discharge-measuring stations and the basin they belong to

In [3]:
#Helper function to query the database and return a pandas dataframe if required.
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host=geonorway, port=5432, database='geonorway') as db:
        cursor = db.cursor()
        cursor.execute(query)
        if fetch:
            result = sqlio.read_sql_query(query, db)
            return result
        
#Schema where the nve data resides
schema = 'nveq'

#Querying the coordinates of the stations and the basin they are in:
sql = '''with bla as
(select stationid,stationname,st_transform(geom,3045) as geom from {}.stations)
select b.stationid,a.gid,a.vassomr,st_area(a.geom)/1e6 as area
from nedborfelt.vassdragsomr as a, bla as b 
where st_intersects(a.geom,b.geom);'''.format(schema)

a = query(sql)
display(a)

,stationid,gid,vassomr,area
0,1.198.0,1,Haldenvassdraget/Iddefjorden,2507.380211
1,1.200.0,1,Haldenvassdraget/Iddefjorden,2507.380211
2,1.48.0,1,Haldenvassdraget/Iddefjorden,2507.380211
3,1.49.0,1,Haldenvassdraget/Iddefjorden,2507.380211
4,1.50.0,1,Haldenvassdraget/Iddefjorden,2507.380211
...,...,...,...,...
597,97.5.0,97,"Storfjorden sør, Hjørundfjorden og Sykkylvsfjo...",868.995031
598,98.4.0,98,"Storfjorden sør, Sunnylvsfjorden og Geirangerf...",1003.641202
599,99.1.0,99,Tafjordvassdraget/Tafjorden og Norddalsfjorden...,643.801323
600,99.17.0,99,Tafjordvassdraget/Tafjorden og Norddalsfjorden...,643.801323


In [4]:
# Checking if all stations fall inside a basin. The query should be empty if all stations fall inside a basin
sql = '''with bla as
(select stationid,stationname,st_transform(geom,3045) as geom from {}.stations)
select b.stationid,b.stationname from bla as b 
where b.stationid not in (select b.stationid from bla as b, nedborfelt.vassdragsomr as a where st_intersects(a.geom,b.geom));'''.format(schema)

display(query(sql))

,stationid,stationname
0,19.244.0,Nye Jørundland


In [5]:
#There is a problem with Nye Jørundland, it appears that the latitude is wrong. This will need fixing
display(query('''select * from nveq.stations where stationid='19.244.0'  '''))

#This query gets the coordinates directly from the metadata that was obtained from HydAPI
a = query('''with bla as
(
select json_array_elements(metadata->'data') as singleelement from nveq.metadata
)
select singleelement->'stationId' as sid, singleelement->'stationName' as name ,singleelement->'parameterName' as parameter,
singleelement->'longitude' as longitude,singleelement->'latitude' as latitude,
singleelement->'utmEast_Z33' as utme,singleelement->'utmNorth_Z33' as utmn
from  bla as a 
where (a.singleelement->>'stationId') = '19.244.0'
''')
display(a)

,sid,stationname,stationid,latitude,longitude,geom
0,241,Nye Jørundland,19.244.0,1.11498,5.61744,0101000020E61000000EF8FC3042781640F41ABB44F5D6...


,sid,name,parameter,longitude,latitude,utme,utmn
0,19.244.0,Nye Jørundland,Vannstand,5.61744,1.11498,-548573,124922
1,19.244.0,Nye Jørundland,Vannføring,5.61744,1.11498,-548573,124922


In [6]:
if not a.empty:
    from pyproj import Transformer
    transformer = Transformer.from_crs("epsg:25833", "epsg:4326")
    new_coord = transformer.transform(a.loc[0,'utme'],a.loc[0,'utmn'])
    display(new_coord)

(1.1149882510510696, 5.617439450536286)

## Creating table with outlet coordinates
A common problem with coordinates is that they do fall on an actual "river". In order for the basin delineation algorithm to work, the given coordinates should fall inside a cell considered a river in the flow accumulation raster. 
In order to guarantee this we will nudge, if necessary, the given coordinates so they fall in a cell considered to be a river. The process is not fool proof and the results should be quality controlled. 

In [7]:
#We will not include the Nye Jørundland stations until we get the right coordinates for it
#Please note that in the outlet table we will setup the srid of the outlet to 3035 since
#that is the epsg of the rasters we use to compute the basins
sql = '''drop table if exists {schema}.outlet cascade;
create table {schema}.outlet as select sid, stationname, stationid, latitude, longitude, st_transform(geom,25833) as geom from {schema}.stations as a
where a.stationid != '19.244.0';
update {schema}.outlet
set latitude=st_y(geom),
longitude=st_x(geom);

create index {schema}_outlet_idx on {schema}.outlet using gist(geom);

delete from {schema}.outlet where latitude > 8e6;

alter table {schema}.outlet add primary key (sid);

alter table {schema}.outlet add constraint fk_basin
foreign key(sid) 
references {schema}.stations(sid);

'''.format(schema=schema)
query(sql,fetch=False)
display(query('''select * from {schema}.outlet order by latitude desc;'''.format(schema=schema)))

,sid,stationname,stationid,latitude,longitude,geom
0,393,Nordmannset,230.1.0,7.911030e+06,9.544695e+05,0101000020E96400003A0C890ACB202D4155F451879D2D...
1,398,Båtsfjord,237.1.0,7.894752e+06,1.041687e+06,0101000020E9640000136099262DCA2F4185C57EF4B71D...
2,397,Kongsfjordelv,236.8.0,7.893933e+06,1.018511e+06,0101000020E9640000DC02FFC01E152F4146493833EB1C...
3,388,Ørretvatn,222.2.0,7.883016e+06,8.849578e+05,0101000020E9640000778DED92BB012B41BC14E6FA4112...
4,380,Kvalsund,213.4.0,7.844730e+06,8.327179e+05,0101000020E9640000FA9F22C09B6929411E6CAD76DEEC...
...,...,...,...,...,...,...
593,399,Møska (Skolandsvatnet),24.8.0,6.474075e+06,3.380677e+04,0101000020E9640000B01E2C88D881E040200F90B256B2...
594,392,Gaupefossen,23.8.0,6.470237e+06,5.120889e+04,0101000020E964000070FB797C1C01E940B36B944797AE...
595,385,Kjølemo,22.4.0,6.466492e+06,6.037900e+04,0101000020E9640000F09C281F607BED40774AC20FEFAA...
596,366,Sømskleiva,21.49.0,6.466472e+06,9.147462e+04,0101000020E96400006C08BDFA2955F640FC1B77FDE9AA...


The NVE river dataset was burned into the elevation raster when doing the hydrological processing. 
Therefore, a good first approximation for the outlet position is the nearest point falling in a river.

In [8]:
%%time
#Moving outlet to the closest river, within a 500m radius
sql = '''update {schema}.outlet as a 
set geom = 
case 
when st_intersects(st_buffer(a.geom,500),b.geom) then
(
select st_closestpoint(st_union(b.geom),a.geom) from elv.elvenett25833 as b 
where st_intersects(st_buffer(a.geom,500),b.geom)
)
else a.geom
end
from elv.elvenett25833 as b where st_intersects(st_buffer(a.geom,500),b.geom);
;
update {schema}.outlet
set latitude=st_y(geom),
longitude=st_x(geom);

'''.format(schema=schema)
query(sql,fetch=False)



CPU times: user 5.32 ms, sys: 1.38 ms, total: 6.7 ms
Wall time: 10.1 s


In [9]:
#Loading the flow accumulation raster locally. We will do the computation one basin at a time. We exclude the stations in Svalbard
#for which we have no elevation data
sql = '''with bla as
(select sid,stationid,stationname,st_transform(geom,3045) as geom from {schema}.stations)
select distinct a.gid, array_agg(sid) as sids
from nedborfelt.vassdragsomr as a, bla as b 
where st_intersects(a.geom,b.geom)
and st_y(b.geom) < 8e6
group by a.gid;'''.format(schema = schema)

a = query(sql)
display(a)

,gid,sids
0,1,"[5, 4, 3, 2, 1]"
1,2,"[336, 338, 339, 340, 313, 312, 311, 310, 309, ..."
2,3,"[434, 433]"
3,5,"[480, 483, 481, 482]"
4,6,"[500, 501, 499, 498, 497, 496, 495, 494, 493, ..."
...,...,...
160,250,[436]
161,254,"[438, 437]"
162,255,[439]
163,258,"[440, 443, 444, 441]"


In [10]:
#Processing all stations within one basin
import numpy as np
from tqdm import tqdm

#Creating temporary directory to download data and place intermediary files
with Connection('localhost') as c:
    c.local('rm -rf geodata && mkdir geodata')

#A river defined as a pixel with more than 1km2  upstream areas (10k cells)
def find_nearest_river(pixel):
    #Returns index to numpy array
    distances = (nonzero[:,0] - pixel[0]) ** 2 + (nonzero[:,1] - pixel[1]) ** 2
    nearest_index = np.argmin(distances)
#     display(nearest_index)
    return nonzero[nearest_index]

def coordsFromPixel(i,j):
    #i, j are pixel coordinated in the numpy array
    long = xoffset + (j+0.5) * px_w  #adding half a pixel width so the coordinates are centered
    lat = yoffset + (i+0.5) * px_h  
    return long,lat

def pixelFromCoord(long,lat):
    #Returns index to numpy array
    i = (long - xoffset) / px_w
    j = (lat - yoffset) / px_h
#     display([sid,long,lat,i,j])
    return (int(j),int(i))

sql = '''select sid, longitude, latitude from {schema}.outlet where sid in ({idList}); '''

updateSql = '''update {schema}.outlet
set longitude = {long},
latitude = {lat}
where sid = {sid};

update {schema}.outlet
set geom = st_setsrid(st_makepoint(longitude,latitude),25833)
where sid={sid};
'''

for i,j in tqdm(a.iterrows()):
    gid = j['gid']
    sids = j['sids']
#     display(gid,sids)    
    coords = query(sql.format(schema=schema,idList=str(sids)[1:-1]))
    coords.set_index('sid',inplace=True)
#     display(coords)
    with Connection(**config) as c:
        c.get('/home/jose-luis/flatLake/basin_{}_flow_acc.tif'.format(gid),'./geodata/')
        dataset = gdal.Open('geodata/basin_{}_flow_acc.tif'.format(gid),gdal.GA_ReadOnly)
        xoffset, px_w, rot1, yoffset, rot2, px_h = dataset.GetGeoTransform()
#         display([xoffset, px_w, rot1, yoffset, rot2, px_h])
        #IMPORTANT: the index order is inverted when loading into a numpy array. 
        #This is the reason of the index gymnastics in the following code
        river = np.array(dataset.GetRasterBand(1).ReadAsArray())
        nonzero = np.argwhere(river > 10000)
#         display(len(nonzero))
        for sid in sids:
            long = coords.loc[sid,'longitude']
            lat = coords.loc[sid,'latitude']
            currentPixel = pixelFromCoord(long,lat)
#             display("Current flow_acc: {}".format(river[currentPixel[1],currentPixel[0]]))
            riverPixel = find_nearest_river(currentPixel)
#             display("Updated flow_acc: {}".format(river[riverPixel[0],riverPixel[1]]))
#             display([long,lat,currentPixel,riverPixel])
            new_x,new_y = coordsFromPixel(riverPixel[0],riverPixel[1])
#             display([sid,long,lat,new_x,new_y])
            query(updateSql.format(schema=schema,sid=sid,long=new_x,lat=new_y),fetch=False)
    with Connection('localhost') as c:
        c.local('rm geodata/basin_{}_flow_acc.tif'.format(gid))
    
#             display([long,lat,new_x,new_y])
            
        0

165it [07:52,  2.86s/it]


## Computing the basins from the modified coordinates

In [17]:
#Finding out the gid number for the flow accumulation raster
sql = '''with bla as
(select sid,stationid,st_transform(geom,3045) as geom from {schema}.stations)
select b.sid,b.stationid,a.gid,a.vassomr,st_area(a.geom)/1e6 as area
from nedborfelt.vassdragsomr as a, bla as b 
where st_intersects(a.geom,b.geom);'''.format(schema=schema)

a = query(sql)
display(a)

query('drop table if exists {schema}.basins;'.format(schema=schema),fetch=False)
query('''create table {schema}.basins(sid integer not  null,area double precision, geom geometry(multipolygon,25833),
primary key (sid),
constraint fk_outlet
foreign key(sid) 
references {schema}.outlet(sid)
);
'''.format(schema=schema),fetch=False)

#Creating folder to store results
with Connection(**config) as c:
    c.run('rm -rf watershed && mkdir watershed')
    

script='''#! /bin/bash
cd watershed
rm -f out.* basin.*

pgsql2shp -f out{sid}.shp geonorway "select sid as id, geom from {schema}.outlet where sid={sid}"
mpiexec -n 3 gagewatershed -p /home/jose-luis/flatLake/basin_{gid}_flow_dir.tif -o out{sid}.shp -gw watershed_{sid}.tif
gdal_polygonize.py -8 -f "ESRI Shapefile" watershed_{sid}.tif basin{sid}.shp
shp2pgsql -d -s 25833 basin{sid}.shp {schema}.temp | psql -d geonorway
echo "insert into {schema}.basins(sid,geom) select dn, st_makevalid(geom) from {schema}.temp;" | psql -d geonorway
'''

for i,j in a.iterrows():   
# for sid in sids:
#     sid=j['station_id']
    display([j['sid'],j['stationid']])
    with Connection(**config) as c:
        with open('script.sh','w') as f:
            cond = {'sid': j['sid'], 'gid': j['gid'], 'schema': schema}
            f.write(script.format(**cond))
        c.put('script.sh')
        c.run('chmod +x script.sh')
        c.run('./script.sh',hide='both')
    

query('update {schema}.basins set area=st_area(geom)'.format(schema=schema),fetch=False);
query('create index {schema}_basins_idx on {schema}.basins using gist(geom);'.format(schema=schema),fetch=False)

,sid,stationid,gid,vassomr,area
0,1,1.198.0,1,Haldenvassdraget/Iddefjorden,2507.380211
1,2,1.200.0,1,Haldenvassdraget/Iddefjorden,2507.380211
2,3,1.48.0,1,Haldenvassdraget/Iddefjorden,2507.380211
3,4,1.49.0,1,Haldenvassdraget/Iddefjorden,2507.380211
4,5,1.50.0,1,Haldenvassdraget/Iddefjorden,2507.380211
...,...,...,...,...,...
597,599,97.5.0,97,"Storfjorden sør, Hjørundfjorden og Sykkylvsfjo...",868.995031
598,600,98.4.0,98,"Storfjorden sør, Sunnylvsfjorden og Geirangerf...",1003.641202
599,601,99.1.0,99,Tafjordvassdraget/Tafjorden og Norddalsfjorden...,643.801323
600,602,99.17.0,99,Tafjordvassdraget/Tafjorden og Norddalsfjorden...,643.801323


[1, '1.198.0']

[2, '1.200.0']

[3, '1.48.0']

[4, '1.49.0']

...

[599, '97.5.0']

[600, '98.4.0']

[601, '99.1.0']

[602, '99.17.0']

[603, '99.2.0']

In [12]:
23131*32196


744725676